# Project EXISTS 2025

## Libraries

In [ ]:
COLAB =True # IF YOU USE GOOGLE COLAB -> COLAB = True
PIP = True # IF YOU NEED INSTALL LIBRARIES -> PIP = True

if PIP:
    !pip install transformers --upgrade
    !pip install datasets accelerate
    !pip install evaluate
    !pip install -U PyEvALL

!pip install torch
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install -U optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 21.9 MB/s eta 0:00:00


In [ ]:
# Standard libraries
import os
import sys
import tempfile
import time
import ast
import json
import random

# Data manipulation
import numpy as np
import pandas as pd

# PyTorch
import torch
from torch.utils.data import Dataset, DataLoader

# Transformers
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)

# PEFT (Parameter-Efficient Fine-Tuning)
from peft import LoraConfig, get_peft_model, TaskType

# Evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Optuna for hyperparameter tuning
import optuna

# PyEvALL for evaluation
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.metrics.metricfactory import MetricFactory
from pyevall.reports.reports import PyEvALLReport
from pyevall.utils.utils import PyEvALLUtils


## Drive and Preloaded functions



In [ ]:
if COLAB is True:
  from google.colab import drive
  drive.mount('/content/drive')
  base_path = "/content/drive/MyDrive"
else:
  base_path = ".."
base_path

library_path = base_path
sys.path.append(library_path)
from readerEXIST2025_2 import EXISTReader

Mounted at /content/drive


In [ ]:
# path to the dataset, adapt this path wherever you have the dataset
dataset_path = os.path.join(base_path, "EXIST_2025_Dataset_V0.3/")

file_train = os.path.join(dataset_path, "EXIST2025_training.json")
file_dev = os.path.join(dataset_path, "EXIST2025_dev.json")
file_test = os.path.join(dataset_path, "EXIST2025_test_clean.json")


reader_train = EXISTReader(file_train)
reader_dev = EXISTReader(file_dev)
reader_test = EXISTReader(file_test)


EnTrainTask1, EnDevTask1, EnTestTask1 = reader_train.get(lang="EN", subtask="1"), reader_dev.get(lang="EN", subtask="1"), reader_test.get(lang="EN", subtask="1")
EnTrainTask2, EnDevTask2, EnTestTask2 = reader_train.get(lang="EN", subtask="2"), reader_dev.get(lang="EN", subtask="2"), reader_test.get(lang="EN", subtask="2")
EnTrainTask3, EnDevTask3, EnTestTask3 = reader_train.get(lang="EN", subtask="3"), reader_dev.get(lang="EN", subtask="3"), reader_test.get(lang="EN", subtask="3")


SpTrainTask1, SpDevTask1, SpTestTask1  = reader_train.get(lang="ES", subtask="1"), reader_dev.get(lang="ES", subtask="1"), reader_test.get(lang="ES", subtask="1")
SpTrainTask2, SpDevTask2, SpTestTask2  = reader_train.get(lang="ES", subtask="2"), reader_dev.get(lang="ES", subtask="2"), reader_test.get(lang="ES", subtask="2")
SpTrainTask3, SpDevTask3, SpTestTask3  = reader_train.get(lang="ES", subtask="3"), reader_dev.get(lang="ES", subtask="3"), reader_test.get(lang="ES", subtask="3")

!ls ../../corpora/EXIST_2025_Dataset_V0.3/

ls: cannot access '../../corpora/EXIST_2025_Dataset_V0.3/': No such file or directory


## Seeding

In [ ]:
def set_seed(seed=1234):
    """
    Sets the seed to make everything deterministic, for reproducibility of experiments
    Parameters:
    seed: the number to set the seed to
    Return: None
    """
    # Random seed
    random.seed(seed)
    # Numpy seed
    np.random.seed(seed)
    # Torch seed
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    # os seed
    os.environ['PYTHONHASHSEED'] = str(seed)


## Clases and Functions

In [ ]:
class SexismDataset(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)
        }

### Evaluation Metrics

In [ ]:
def compute_metrics_1(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='binary', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def compute_metrics_2(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def compute_metrics_3(pred, lencoder):
    labels = pred.label_ids
    #preds = pred.predictions.argmax(-1)
    preds = torch.sigmoid(torch.tensor(pred.predictions)).numpy()
    preds_binary = (preds >= 0.5).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds_binary, average=None, zero_division=0
    )
    acc = accuracy_score(labels, preds_binary)
    icm= ICMWrapper(lencoder.inverse_transform(preds_binary), lencoder.inverse_transform(labels), multi=True)
    # Macro averages
    precision_macro = np.mean(precision)
    recall_macro = np.mean(recall)
    f1_macro = np.mean(f1)
    metrics = {}
    metrics.update({
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'ICM': icm
    })
    return metrics

def ICMWrapper(pred, labels, multi=False,ids=None):
    test = PyEvALLEvaluation()
    metrics=[MetricFactory.ICM.value]
    params= dict()
    fillLabel=None
    if multi:
        params[PyEvALLUtils.PARAM_REPORT]="embedded"
        hierarchy={"True":['IDEOLOGICAL-INEQUALITY', 'STEREOTYPING-DOMINANCE', 'MISOGYNY-NON-SEXUAL-VIOLENCE', 'OBJECTIFICATION', 'SEXUAL-VIOLENCE'],
        "False":[]}
        params[PyEvALLUtils.PARAM_HIERARCHY]=hierarchy
        fillLabel = lambda x: ["False"] if len(x)== 0 else x
    else:
        params[PyEvALLUtils.PARAM_REPORT]="simple"
        fillLabel = lambda x: str(x)


    truth_name, predict_name=None, None
    if ids is None:
        ids=list(range(len(labels)))

    with tempfile.NamedTemporaryFile(mode='w', delete=False, encoding='utf-8') as truth:
        truth_name=truth.name
        truth_df=pd.DataFrame({'test_case': ['EXIST2025']*len(labels),
                        'id': [str(x) for x in ids],
                        'value': [fillLabel(x) for x in labels]})
        if multi==True:
            truth_df=truth_df.astype('object')
        truth.write(truth_df.to_json(orient="records"))

    with  tempfile.NamedTemporaryFile(mode='w', delete=False) as predict:
        predict_name=predict.name
        predict_df=pd.DataFrame({'test_case': ['EXIST2025']*len(pred),
                        'id': [str(x) for x in ids],
                        'value': [fillLabel(x) for x in pred]})
        if multi==True:
            predict_df=predict_df.astype('object')
        predict.write(predict_df.to_json(orient="records"))

    report = test.evaluate(predict_name, truth_name, metrics, **params)
    os.unlink(truth_name)
    os.unlink(predict_name)

    icm = None
    if 'metrics' in report.report:
        if 'ICM' in report.report["metrics"]: icm=float(report.report["metrics"]['ICM']["results"]["average_per_test_case"])
    return icm


In [ ]:
######################################CHANGE###############################################
from peft import LoraConfig, get_peft_model, TaskType
###########################################################################################

def sexism_classification_pipeline_task1_LoRA(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=2, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc = LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )

    ######################################CHANGE###############################################
    # Configure LoRA
    lora_config = LoraConfig(
    task_type= args.get("task_type", TaskType.SEQ_CLS),
    target_modules= args.get("target_modules", ["query", "value"]),
    r= args.get("rank", 64),  # Rank of LoRA adaptation
    lora_alpha=args.get("lora_alpha", 32),  # Scaling factor
    lora_dropout=args.get("lora_dropout", 0.1),
    bias=args.get("bias", "none")
)
    ###########################################################################################

    ######################################CHANGE###############################################
    # Prepare LoRA model
    peft_model = get_peft_model(model, lora_config)

    ###########################################################################################
    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results_task1_LoRA0'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"f1")
    )

    # Initialize Trainer
    trainer = Trainer(
        ######################################CHANGE###############################################
        # Prepare LoRA model
        model=peft_model,
        ###########################################################################################
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_1,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    ######################################CHANGE###############################################
    #Saving the new weigths for the LoRA model
    trainer.save_model('./final_best_model_LoRA')
    # Notice that, in this case only the LoRA matrices are saved.
    # The weigths for the classification head are not saved.
    ###########################################################################################

    ######################################CHANGE###############################################
    #Mixing the LoRA matrices with the weigths of the base model used
    mixModel=peft_model.merge_and_unload()
    mixModel.save_pretrained("./final_best_model_mixpeft")
    # IN this case the full model is saved.
    ###########################################################################################

    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)
        })
        submission_df.to_csv('sexism_predictions_task1.csv', index=False)
        print("Prediction for TASK 1 completed. Results saved to sexism_predictions_task1.csv")
        return mixModel, submission_df
    return mixModel, eval_results



def sexism_classification_pipeline_task2_LoRA(trainInfo, devInfo, testInfo=None, model_name='roberta-base', nlabels=4, ptype="single_label_classification", **args):
    # Model and Tokenizer
    labelEnc = LabelEncoder()
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )
    ######################################CHANGE###############################################
    # Configure LoRA
    lora_config = LoraConfig(
    task_type= args.get("task_type", TaskType.SEQ_CLS),
    target_modules= args.get("target_modules", ["query", "value"]),
    r= args.get("rank", 64),  # Rank of LoRA adaptation
    lora_alpha=args.get("lora_alpha", 32),  # Scaling factor
    lora_dropout=args.get("lora_dropout", 0.1),
    bias=args.get("bias", "none"),
)
    ###########################################################################################

    ######################################CHANGE###############################################
    # Prepare LoRA model
    peft_model = get_peft_model(model, lora_config)

    ###########################################################################################

    # Prepare datasets
    train_dataset = SexismDataset(trainInfo[1], labelEnc.fit_transform(trainInfo[2]),[int(x) for x in trainInfo[0]], tokenizer )
    val_dataset = SexismDataset(devInfo[1], labelEnc.transform(devInfo[2]), [int(x) for x in devInfo[0]], tokenizer)

    # Training Arguments
    training_args = TrainingArguments(
        report_to="none", # alt: "wandb", "tensorboard" "comet_ml" "mlflow" "clearml"
        output_dir= args.get('output_dir', './results_task2_LoRA0'),
        num_train_epochs= args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay',0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy','epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model',"f1") # F1 para el concurso
    )

    # Initialize Trainer
    trainer = Trainer(
        ######################################CHANGE###############################################
        # Prepare LoRA model
        model=peft_model,
        ###########################################################################################
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_2,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience",3))]
    )

    # Fine-tune the model
    trainer.train()

    # Evaluate on validation set
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    ######################################CHANGE###############################################
    #Saving the new weigths for the LoRA model
    trainer.save_model('./final_best_model_LoRA_2')
    # Notice that, in this case only the LoRA matrices are saved.
    # The weigths for the classification head are not saved.
    ###########################################################################################

    ######################################CHANGE###############################################
    #Mixing the LoRA matrices with the weigths of the base model used
    mixModel=peft_model.merge_and_unload()
    mixModel.save_pretrained("./final_best_model_mixpeft_2")
    # IN this case the full model is saved.
    ###########################################################################################

    if testInfo is not None:
        # Prepare test dataset for prediction
        test_dataset = SexismDataset(testInfo[1], [0] * len(testInfo[1]),  [int(x) for x in testInfo[0]],   tokenizer)

        # Predict test set labels
        predictions = trainer.predict(test_dataset)
        predicted_labels = np.argmax(predictions.predictions, axis=1)

        # Create submission DataFrame
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            "test_case": ["EXIST2025"]*len(predicted_labels)
        })
        submission_df.to_csv('sexism_predictions_task2.csv', index=False)
        print("Prediction for TASK 2 completed. Results saved to sexism_predictions_task1.csv")
        return mixModel, submission_df
    return mixModel, eval_results


def sexism_classification_pipeline_task3_LoRA(trainInfo, devInfo, testInfo=None,
                                              model_name='roberta-base',
                                              nlabels=6,
                                              ptype="multi_label_classification",
                                              **args):
    """
    Pipeline para clasificación multi-label (TASK 3) con adaptación LoRA.

    Parámetros:
      - trainInfo, devInfo, testInfo: estructuras con información, donde se espera que:
          trainInfo = (lista_ids, lista_textos, lista_etiquetas)
          devInfo   = (lista_ids, lista_textos, lista_etiquetas)
          testInfo  = (lista_ids, lista_textos, _dummy_)  [para test se generan etiquetas dummy]
      - model_name: nombre (o path) del modelo base.
      - nlabels: número de etiquetas.
      - ptype: tipo de problema para el modelo ('multi_label_classification').
      - **args: permite ajustar hiperparámetros (por ejemplo, learning_rate, num_train_epochs, etc.) y la configuración LoRA.

    Retorna:
      - mixModel: modelo final (con LoRA fusionado con los pesos base).
      - submission_df o eval_results: según se proporcione o no testInfo.
    """
    # Codificador de etiquetas para multi-label
    labelEnc = MultiLabelBinarizer()


    # Modelo y Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    base_model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=nlabels,
        problem_type=ptype
    )

    ###################################### CONFIGURACIÓN LoRA ###############################################
    lora_config = LoraConfig(
        task_type=args.get("task_type", TaskType.SEQ_CLS),
        target_modules=args.get("target_modules", ["query", "value"]),
        r=args.get("rank", 64),                # Rango de adaptación LoRA
        lora_alpha=args.get("lora_alpha", 32),  # Factor de escalado LoRA
        lora_dropout=args.get("lora_dropout", 0.1),
        bias=args.get("bias", "none")
    )
    # Aplicar LoRA al modelo base
    peft_model = get_peft_model(base_model, lora_config)
    #######################################################################################################

    # Preparación de datasets usando SexismDatasetMulti para clasificación multi-label
    train_dataset = SexismDatasetMulti(
        texts=trainInfo[1],
        labels=labelEnc.fit_transform(trainInfo[2]),
        ids=[int(x) for x in trainInfo[0]],
        tokenizer=tokenizer
    )
    val_dataset = SexismDatasetMulti(
        texts=devInfo[1],
        labels=labelEnc.transform(devInfo[2]),
        ids=[int(x) for x in devInfo[0]],
        tokenizer=tokenizer
    )

    # Configuración de los argumentos de entrenamiento
    training_args = TrainingArguments(
        report_to="none",  # Alternativas: "wandb", "tensorboard", etc.
        output_dir=args.get('output_dir', './results_task3_LoRA'),
        num_train_epochs=args.get('num_train_epochs', 5),
        learning_rate=args.get('learning_rate', 5e-5),
        per_device_train_batch_size=args.get('per_device_train_batch_size', 16),
        per_device_eval_batch_size=args.get('per_device_eval_batch_size', 64),
        warmup_steps=args.get('warmup_steps', 500),
        weight_decay=args.get('weight_decay', 0.01),
        logging_dir=args.get('logging_dir', './logs'),
        logging_steps=args.get('logging_steps', 10),
        eval_strategy=args.get('eval_strategy', 'epoch'),
        save_strategy=args.get('save_strategy', "epoch"),
        save_total_limit=args.get('save_total_limit', 1),
        load_best_model_at_end=args.get('load_best_model_at_end', True),
        metric_for_best_model=args.get('metric_for_best_model', "ICM")
    )

    # Inicialización del Trainer con función de métricas propia para multi-label (compute_metrics_3)
    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=partial(compute_metrics_3, lencoder=labelEnc),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=args.get("early_stopping_patience", 3))]
    )

    # Entrenamiento
    trainer.train()

    # Evaluación en el conjunto de validación
    eval_results = trainer.evaluate()
    print("Validation Results:", eval_results)

    ###################################### GUARDADO DEL MODELO ###############################################
    # Se guardan los pesos de LoRA (solo las matrices LoRA)
    trainer.save_model('./final_best_model_LoRA')
    # Se realiza la fusión de las matrices LoRA con los pesos base para obtener el modelo completo
    mixModel = peft_model.merge_and_unload()
    mixModel.save_pretrained("./final_best_model_mixpeft_3")
    #######################################################################################################

    if testInfo is not None:
        # Preparación del dataset de test (se crean etiquetas dummy, pues en test solo se hacen predicciones)
        test_dataset = SexismDatasetMulti(
            texts=testInfo[1],
            labels=[[0] * nlabels for _ in range(len(testInfo[1]))],
            ids=[int(x) for x in testInfo[0]],
            tokenizer=tokenizer
        )

        # Realizar predicciones
        predictions = trainer.predict(test_dataset)
        # Se aplican la función sigmoide y se define un umbral (0.5) para obtener las etiquetas finales
        predicted_probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
        predicted_labels = (predicted_probs >= 0.5).astype(int)

        # Creación del DataFrame para la submission
        submission_df = pd.DataFrame({
            'id': testInfo[0],
            'label': labelEnc.inverse_transform(predicted_labels),
            'test_case': ["EXIST2025"] * len(predicted_labels)
        })
        submission_df.to_csv('sexism_predictions_task3_LoRA.csv', index=False)
        print("Prediction for TASK 3 (LoRA) completed. Results saved to sexism_predictions_task3_LoRA.csv")
        return mixModel, submission_df

    return mixModel, eval_results

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.preprocessing import LabelEncoder

def run_lora_experiments(
    task_num: int,
    model_names: list,
    params: dict,
    trainInfo,
    devInfo,
    testInfo=None
) -> pd.DataFrame:
    """
    Runs a batch of LoRA fine-tuning experiments for task 1, 2 or 3.

    Args:
        task_num:       1, 2 or 3 – which sexism_classification_pipeline_taskX_LoRA to call.
        model_names:    List of HF model names (strings).
        params:         Dict of hyperparameters to pass as **kwargs.
        trainInfo:      Your train tuple (ids, texts, labels).
        devInfo:        Your dev tuple (ids, texts, labels).
        testInfo:       Optional test tuple (ids, texts) for predictions.

    Returns:
        DataFrame with one row per model, columns = ['model', *eval_metrics..., 'epoch'].
    """
    # map task number → (pipeline_fn, nlabels, problem_type)
    pipeline_map = {
        1: (sexism_classification_pipeline_task1_LoRA, 2, "single_label_classification"),
        2: (sexism_classification_pipeline_task2_LoRA, 4, "single_label_classification"),
        3: (sexism_classification_pipeline_task3_LoRA, 6, "multi_label_classification"),
    }
    if task_num not in pipeline_map:
        raise ValueError(f"Unsupported task {task_num}. Choose 1, 2 or 3.")

    pipeline_fn, nlabels, ptype = pipeline_map[task_num]
    metrics_list = []

    for model_name in model_names:
        print(f"→ Running task{task_num} LoRA on {model_name!r} …")
        # pipeline returns (model, eval_results) if testInfo is None
        _, eval_results = pipeline_fn(
            trainInfo,
            devInfo,
            testInfo,
            model_name,
            nlabels,
            ptype,
            **params
        )
        # eval_results is a dict like {'eval_accuracy':…, 'eval_f1':…, …, 'epoch':…}
        row = {"model": model_name}
        row.update(eval_results)
        metrics_list.append(row)

    df = pd.DataFrame(metrics_list)
    # optional: reorder cols so 'model' comes first
    cols = ["model"] + [c for c in df.columns if c != "model"]
    return df[cols]


def select_best_model(df: pd.DataFrame, task_num: int) -> str:
    """
    Given the DataFrame of metrics (as returned by run_lora_experiments),
    computes a composite 'score' depending on task and returns the best model name.
    """
    if task_num == 1:
        df["score"] = 0.6 * df["eval_f1"] + 0.4 * df["eval_accuracy"]
    elif task_num == 2:
        df["score"] = (
            0.5 * df["eval_f1"] +
            0.3 * df["eval_accuracy"] +
            0.2 * df["eval_precision"]
        )
    elif task_num == 3:
        df["score"] = (
            0.4 * df["eval_micro_f1"] +
            0.4 * df["eval_macro_f1"] +
            0.2 * df["eval_subset_accuracy"]
        )
    else:
        raise ValueError(f"Unknown task {task_num}")

    best_row = df.sort_values("score", ascending=False).iloc[0]
    print(f"→ Winning model for task{task_num}: {best_row['model']} (score={best_row['score']:.4f})")
    return best_row["model"]


def optimize_lora_hyperparams(
    task_num: int,
    best_model_name: str,
    params_base: dict,
    trainInfo,
    devInfo,
    n_trials: int = 20
) -> dict:
    """
    Runs an Optuna search to tune LoRA hyperparameters for the given task
    and model checkpoint. Returns the optimized params dict.

    Args:
        task_num:         1, 2 or 3 – selects number of labels & compute_metrics.
        best_model_name:  HF model checkpoint chosen after initial comparison.
        params_base:      Base params dict (num_train_epochs, batch_sizes, etc.).
        trainInfo:        Tuple (ids, texts, labels) for training.
        devInfo:          Tuple (ids, texts, labels) for validation.
        n_trials:         Number of Optuna trials to run.
    """
    # 1) Map task → (nlabels, compute_metrics_fn)
    compute_map = {
        1: (2, compute_metrics_1),
        2: (4, compute_metrics_2),
        3: (6, compute_metrics_3),
    }
    if task_num not in compute_map:
        raise ValueError("task_num must be 1, 2, or 3")
    nlabels, compute_metrics = compute_map[task_num]

    # 2) Prepare tokenizer, label encoder, and PyTorch datasets
    tokenizer = AutoTokenizer.from_pretrained(best_model_name)
    label_enc = LabelEncoder()
    y_train = label_enc.fit_transform(trainInfo[2])
    y_dev   = label_enc.transform(devInfo[2])

    train_dataset = SexismDataset(
        texts=trainInfo[1],
        labels=y_train,
        ids=[int(x) for x in trainInfo[0]],
        tokenizer=tokenizer
    )
    eval_dataset = SexismDataset(
        texts=devInfo[1],
        labels=y_dev,
        ids=[int(x) for x in devInfo[0]],
        tokenizer=tokenizer
    )

    # 3) Define Optuna objective
    def objective(trial):
        # 3.1) Suggest hyperparameters
        lr         = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
        r          = trial.suggest_int("r", 8, 128, log=True)
        lora_alpha = trial.suggest_int("lora_alpha", 8, 64, log=True)

        # 3.2) Load base model & apply LoRA
        base_model = AutoModelForSequenceClassification.from_pretrained(
            best_model_name, num_labels=nlabels
        )
        lora_cfg = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            target_modules=["query", "value"],
            r=r,
            lora_alpha=lora_alpha,
            lora_dropout=0.1,
            bias="none"
        )
        peft_model = get_peft_model(base_model, lora_cfg)

        # 3.3) Quick-training arguments
        args = TrainingArguments(
            output_dir=f"optuna_trial_{trial.number}",
            num_train_epochs=10,
            per_device_train_batch_size=params_base["per_device_train_batch_size"],
            per_device_eval_batch_size=params_base["per_device_eval_batch_size"],
            learning_rate=lr,
            logging_steps=params_base.get("logging_steps", 100),
            eval_strategy="epoch",
            save_strategy="no",
            load_best_model_at_end=False,
            report_to="none"
        )

        # 3.4) Trainer
        trainer = Trainer(
            model=peft_model,
            args=args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics
        )

        # 3.5) Train & evaluate
        trainer.train()
        res = trainer.evaluate()
        return res["eval_f1"]

    # 4) Run Optuna study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)
    best_params = study.best_params
    print("✅ Optimized LoRA hyperparameters:", best_params)

    # 5) Merge with base params and return
    optimized_params = params_base.copy()
    optimized_params.update(best_params)
    return optimized_params


set_seed()


# Model Evaluation

# Task 1

In [ ]:
import numpy as np
import optuna
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.preprocessing import LabelEncoder

model_names = [
    "cardiffnlp/twitter-roberta-base-2022-154m",
    "cardiffnlp/twitter-roberta-large-2022-154m",
    "cardiffnlp/twitter-xlm-roberta-base",
    "cardiffnlp/twitter-roberta-base",
    "sdadas/xlm-roberta-large-twitter",
    "g8a9/distilroberta-base-twitter-16M_aug-oct22",
    "andrea-t94/roberta-fine-tuned-twitter",
    "bdotloh/twitter-roberta-base-finetuned-twitter-user-desc"
]

params_twitter_roberta = {
    "num_train_epochs": 100,
    "learning_rate": 0.001,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 8,
    "logging_steps": 100,
}

# Suponiendo que ya tienes EnTrainTask2, EnDevTask2 definidos:
df_metrics = run_lora_experiments(
    task_num=1,
    model_names=model_names,
    params=params_twitter_roberta,
    trainInfo=EnTrainTask1,
    devInfo=EnDevTask1,
    testInfo=None
)

print(df_metrics)
best_model_name = select_best_model(df_metrics, 1)



→ Running task1 LoRA on 'cardiffnlp/twitter-roberta-base-2022-154m' …


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.474800,0.376046,0.840090,0.825553,0.788732,0.865979
2,0.455900,0.370179,0.819820,0.809524,0.752212,0.876289
3,0.400500,0.529913,0.842342,0.830097,0.784404,0.881443
4,0.362200,0.449392,0.846847,0.822917,0.831579,0.814433
5,0.322900,0.567762,0.860360,0.835979,0.858696,0.814433
6,0.386300,0.650543,0.824324,0.782123,0.853659,0.721649
7,0.304700,0.864531,0.806306,0.754286,0.846154,0.680412
8,0.288700,0.920907,0.849099,0.832080,0.809756,0.855670


Validation Results: {'eval_loss': 0.5677616000175476, 'eval_accuracy': 0.8603603603603603, 'eval_f1': 0.8359788359788359, 'eval_precision': 0.8586956521739131, 'eval_recall': 0.8144329896907216, 'eval_runtime': 3.1745, 'eval_samples_per_second': 139.866, 'eval_steps_per_second': 17.641, 'epoch': 8.0}
→ Running task1 LoRA on 'cardiffnlp/twitter-roberta-large-2022-154m' …


tokenizer_config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bdotloh/twitter-roberta-base-finetuned-twitter-user-desc and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.516000,0.405331,0.819820,0.811321,0.747826,0.886598
2,0.493400,0.392478,0.835586,0.817043,0.795122,0.840206
3,0.424800,0.487659,0.801802,0.744186,0.853333,0.659794
4,0.423500,0.436317,0.826577,0.794667,0.823204,0.768041
5,0.388700,0.467698,0.826577,0.812652,0.769585,0.860825


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Validation Results: {'eval_loss': 0.39247798919677734, 'eval_accuracy': 0.8355855855855856, 'eval_f1': 0.8170426065162907, 'eval_precision': 0.7951219512195122, 'eval_recall': 0.8402061855670103, 'eval_runtime': 3.2042, 'eval_samples_per_second': 138.569, 'eval_steps_per_second': 17.477, 'epoch': 5.0}
                                               model  eval_loss  \
0          cardiffnlp/twitter-roberta-base-2022-154m   0.567762   
1         cardiffnlp/twitter-roberta-large-2022-154m   0.694174   
2                cardiffnlp/twitter-xlm-roberta-base   0.408927   
3                    cardiffnlp/twitter-roberta-base   0.442068   
4                   sdadas/xlm-roberta-large-twitter   0.456708   
5      g8a9/distilroberta-base-twitter-16M_aug-oct22   0.416778   
6              andrea-t94/roberta-fine-tuned-twitter   0.419617   
7  bdotloh/twitter-roberta-base-finetuned-twitter...   0.392478   

   eval_accuracy   eval_f1  eval_precision  eval_recall  eval_runtime  \
0       0.860360  0.

In [ ]:
optimized = optimize_lora_hyperparams(
    task_num=1,
    best_model_name=best_model_name,
    params_base=params_twitter_roberta,
    trainInfo=EnTrainTask1,
    devInfo=EnDevTask1,
    n_trials=20
)


print(optimized)



[I 2025-05-14 17:19:34,632] A new study created in memory with name: no-name-825b0510-c881-4733-a5f1-04c2e5a8c915
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.449900,0.421917,0.826577,0.770149,0.914894,0.664948
2,0.391900,0.335344,0.862613,0.845570,0.830846,0.860825
3,0.330300,0.408464,0.846847,0.830000,0.805825,0.855670
4,0.256400,0.424464,0.862613,0.840731,0.851852,0.829897
5,0.210700,0.639807,0.844595,0.809917,0.869822,0.757732
6,0.119800,0.666284,0.876126,0.865526,0.823256,0.912371
7,0.148800,0.676318,0.851351,0.829897,0.829897,0.829897
8,0.131900,0.758225,0.855856,0.835052,0.835052,0.835052
9,0.114200,0.772934,0.858108,0.838875,0.832487,0.845361
10,0.046800,0.869819,0.858108,0.837209,0.839378,0.835052


[I 2025-05-14 17:27:52,517] Trial 0 finished with value: 0.8372093023255814 and parameters: {'learning_rate': 0.0002700550492566597, 'r': 22, 'lora_alpha': 53}. Best is trial 0 with value: 0.8372093023255814.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.457300,0.395979,0.808559,0.784810,0.771144,0.798969
2,0.418800,0.366050,0.828829,0.806122,0.797980,0.814433
3,0.416300,0.381596,0.826577,0.802057,0.800000,0.804124
4,0.367400,0.369941,0.833333,0.800000,0.840909,0.762887
5,0.335200,0.427514,0.833333,0.793296,0.865854,0.731959
6,0.309100,0.364893,0.853604,0.839506,0.805687,0.876289
7,0.357400,0.398514,0.846847,0.818182,0.850000,0.788660
8,0.313600,0.398027,0.849099,0.823219,0.843243,0.804124
9,0.307200,0.394097,0.849099,0.825974,0.832461,0.819588
10,0.258400,0.397791,0.851351,0.828125,0.836842,0.819588


[I 2025-05-14 17:36:04,764] Trial 1 finished with value: 0.828125 and parameters: {'learning_rate': 6.654336600427276e-05, 'r': 8, 'lora_alpha': 25}. Best is trial 0 with value: 0.8372093023255814.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


...

[I 2025-05-14 18:26:03,166] Trial 7 finished with value: 0.7789473684210526 and parameters: {'learning_rate': 1.246998779453707e-05, 'r': 13, 'lora_alpha': 13}. Best is trial 2 with value: 0.8447837150127226.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.435500,0.412040,0.826577,0.768769,0.920863,0.659794
2,0.401400,0.323669,0.849099,0.832080,0.809756,0.855670
3,0.331000,0.359920,0.844595,0.832117,0.788018,0.881443
4,0.284400,0.391969,0.860360,0.833333,0.870787,0.798969


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.435500,0.412040,0.826577,0.768769,0.920863,0.659794
2,0.401400,0.323669,0.849099,0.832080,0.809756,0.855670
3,0.331000,0.359920,0.844595,0.832117,0.788018,0.881443
4,0.284400,0.391969,0.860360,0.833333,0.870787,0.798969
5,0.232700,0.534896,0.849099,0.816438,0.871345,0.768041
6,0.171900,0.538717,0.853604,0.847775,0.776824,0.932990


[W 2025-05-14 18:31:00,625] Trial 8 failed with parameters: {'learning_rate': 0.0003633966492908565, 'r': 21, 'lora_alpha': 12} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-12-e8f4e731f66e>", line 188, in objective
    trainer.train()
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 2245, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/trainer.py", line 2565, in _inner_training_loop
    and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt
[W 2025-05-14 18:31:00,627] Trial 8 failed with value None.


KeyboardInterrupt: 

In [ ]:
final_model, final_metrics = sexism_classification_pipeline_task1_LoRA(
    EnTrainTask1, EnDevTask1, None,
    best_model_name, 2, "single_label_classification",
    **optimized
)

NameError: name 'optimized' is not defined

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. Simple Dataset returning id + tokenized inputs
class EnTestDataset(Dataset):
    def __init__(self, ids, texts, tokenizer, max_length=128):
        self.ids = ids
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        tweet_id = self.ids[idx]
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in encoding.items()}
        item["id"] = tweet_id
        return item

# 2. Load tokenizer + model
modelname = best_model_name
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForSequenceClassification.from_pretrained(
    "./final_best_model_mixpeft",
    num_labels=2
)
model.eval()
model.cuda()

# 3. Prepare dataset & dataloader
ids_list, texts_list = EnTestTask1[0], EnTestTask1[1]
dataset = EnTestDataset(ids_list, texts_list, tokenizer)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# 4. Inference: collect both hard and soft
hard_results = []
soft_results = []
softmax = torch.nn.Softmax(dim=-1)

with torch.no_grad():
    for batch in loader:
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Get probabilities and hard predictions BEFORE tolist()
        probs_tensor = softmax(outputs.logits)               # [B, 2] Tensor
        preds_tensor = outputs.logits.argmax(dim=-1)         # [B] Tensor

        probs = probs_tensor.cpu().tolist()                  # list of [p_no, p_yes]
        preds = preds_tensor.cpu().tolist()                  # list of 0/1 ints

        for tweet_id, prob_vec, pred in zip(batch["id"], probs, preds):
            hard_results.append({
                "test_case": "EXIST2025",
                "id": tweet_id,
                "value": "YES" if pred == 1 else "NO"
            })
            soft_results.append({
                "test_case": "EXIST2025",
                "id": tweet_id,
                "value": {
                    "NO": prob_vec[0],
                    "YES": prob_vec[1]
                }
            })

# 5. Save both JSONs
name = modelname.split("/")[-1]
hard_path = f"predictions_{name}_hard_EnTestTask1.json"
soft_path = f"predictions_{name}_soft_EnTestTask1.json"

with open(hard_path, "w", encoding="utf-8") as f:
    json.dump(hard_results, f, indent=4, ensure_ascii=False)

with open(soft_path, "w", encoding="utf-8") as f:
    json.dump(soft_results, f, indent=4, ensure_ascii=False)

print(f"Saved {len(hard_results)} hard predictions to {hard_path}")
print(f"Saved {len(soft_results)} soft predictions to {soft_path}")




# Task 2

In [ ]:
import numpy as np
import optuna
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.preprocessing import LabelEncoder

model_names = [
    "cardiffnlp/twitter-roberta-base-2022-154m",
    "cardiffnlp/twitter-roberta-large-2022-154m",
    "cardiffnlp/twitter-xlm-roberta-base",
    "cardiffnlp/twitter-roberta-base",
    "sdadas/xlm-roberta-large-twitter",
    "g8a9/distilroberta-base-twitter-16M_aug-oct22",
    "andrea-t94/roberta-fine-tuned-twitter",
    "bdotloh/twitter-roberta-base-finetuned-twitter-user-desc"
]

params_twitter_roberta = {
    "num_train_epochs": 100,
    "learning_rate": 0.001,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 8,
    "logging_steps": 100,
}

# Suponiendo que ya tienes EnTrainTask2, EnDevTask2 definidos:
df_metrics = run_lora_experiments(
    task_num=2,
    model_names=model_names,
    params=params_twitter_roberta,
    trainInfo=EnTrainTask2,
    devInfo=EnDevTask2,
    testInfo=None
)


print(df_metrics)
best_model_name = select_best_model(df_metrics, 2)
print(f"→ Modelo ganador: {best_model_name}")



→ Running task2 LoRA on 'cardiffnlp/twitter-roberta-base-2022-154m' …


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.734100,0.752999,0.752525,0.385108,0.350000,0.433882
2,0.706400,0.840303,0.752525,0.457245,0.447533,0.480665
3,0.682100,0.729376,0.765152,0.429229,0.605350,0.446374
4,0.578100,0.747790,0.762626,0.505960,0.495163,0.552995
5,0.522400,0.728120,0.752525,0.487903,0.473701,0.535091
6,0.434100,0.834108,0.752525,0.519301,0.532310,0.516001
7,0.408200,0.849149,0.744949,0.524876,0.519016,0.539880
8,0.414300,0.842113,0.777778,0.549166,0.560222,0.541670
9,0.479700,0.984255,0.755051,0.496739,0.572351,0.494908
10,0.340000,0.875209,0.767677,0.538091,0.591647,0.522589


Validation Results: {'eval_loss': 0.8421133756637573, 'eval_accuracy': 0.7777777777777778, 'eval_f1': 0.5491663680739036, 'eval_precision': 0.5602221431380996, 'eval_recall': 0.5416697224344283, 'eval_runtime': 2.8623, 'eval_samples_per_second': 138.35, 'eval_steps_per_second': 17.468, 'epoch': 11.0}
→ Running task2 LoRA on 'cardiffnlp/twitter-roberta-large-2022-154m' …


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-large-2022-154m and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.751900,0.753085,0.737374,0.367639,0.336681,0.406529
2,0.724700,0.848943,0.696970,0.404440,0.425353,0.444009
3,0.678000,0.753751,0.755051,0.398796,0.483965,0.422046
4,0.599900,0.836626,0.712121,0.419165,0.413240,0.435863
5,0.596900,0.719480,0.722222,0.440897,0.421309,0.466287
6,0.499500,0.760065,0.752525,0.520517,0.549982,0.507472
7,0.501000,0.951712,0.737374,0.502643,0.545909,0.499379
8,0.501500,0.815754,0.747475,0.484119,0.532126,0.488098
9,0.478700,0.793516,0.775253,0.544136,0.599276,0.525341
10,0.443700,1.035407,0.752525,0.472921,0.693973,0.494017


Validation Results: {'eval_loss': 0.9308944940567017, 'eval_accuracy': 0.7525252525252525, 'eval_f1': 0.5644561586576232, 'eval_precision': 0.5714502381815834, 'eval_recall': 0.5693836261777439, 'eval_runtime': 1.5266, 'eval_samples_per_second': 259.399, 'eval_steps_per_second': 32.752, 'epoch': 15.0}
→ Running task2 LoRA on 'bdotloh/twitter-roberta-base-finetuned-twitter-user-desc' …


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at bdotloh/twitter-roberta-base-finetuned-twitter-user-desc and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.754200,0.759485,0.727273,0.356658,0.329324,0.388941
2,0.744000,0.757709,0.724747,0.438173,0.440476,0.461148
3,0.715100,0.682513,0.755051,0.414585,0.471605,0.435198
4,0.607600,0.748496,0.732323,0.468958,0.456678,0.510809
5,0.537400,0.753089,0.744949,0.465837,0.477781,0.473346
6,0.485100,0.781956,0.744949,0.464195,0.449641,0.481426
7,0.527700,0.933123,0.752525,0.497683,0.550299,0.516838
8,0.529600,0.744259,0.752525,0.469116,0.459911,0.487993
9,0.471100,0.740664,0.747475,0.522586,0.536551,0.514304
10,0.437100,0.834895,0.757576,0.454400,0.683406,0.464630


Validation Results: {'eval_loss': 0.740664005279541, 'eval_accuracy': 0.7474747474747475, 'eval_f1': 0.5225862372921196, 'eval_precision': 0.5365513311165485, 'eval_recall': 0.5143041762159409, 'eval_runtime': 2.9251, 'eval_samples_per_second': 135.378, 'eval_steps_per_second': 17.093, 'epoch': 12.0}
                                               model  eval_loss  \
0          cardiffnlp/twitter-roberta-base-2022-154m   0.842113   
1         cardiffnlp/twitter-roberta-large-2022-154m   0.695258   
2                cardiffnlp/twitter-xlm-roberta-base   0.830945   
3                    cardiffnlp/twitter-roberta-base   0.753783   
4                   sdadas/xlm-roberta-large-twitter   0.862395   
5      g8a9/distilroberta-base-twitter-16M_aug-oct22   0.859861   
6              andrea-t94/roberta-fine-tuned-twitter   0.930894   
7  bdotloh/twitter-roberta-base-finetuned-twitter...   0.740664   

   eval_accuracy   eval_f1  eval_precision  eval_recall  eval_runtime  \
0       0.777778  0.5

In [ ]:
import numpy as np
import optuna
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.preprocessing import LabelEncoder

params_twitter_roberta = {
    "num_train_epochs": 100,
    "learning_rate": 0.001,
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 8,
    "logging_steps": 100,
}

best_model_name = "cardiffnlp/twitter-roberta-base"

optimized_2 = optimize_lora_hyperparams(
    task_num=2,
    best_model_name=best_model_name,
    params_base=params_twitter_roberta,
    trainInfo=EnTrainTask2,
    devInfo=EnDevTask2,
    n_trials=20
)


print(optimized_2)
# [I 2025-05-13 18:08:16,826] Trial 4 finished with value: 0.5675971449024997 and parameters: {'learning_rate': 0.0003822078857884255, 'r': 11, 'lora_alpha': 29}. Best is trial 4 with value: 0.5675971449024997.
#[I 2025-05-13 20:24:40,021] Trial 4 finished with value: 0.5597586424127885 and parameters: {'learning_rate': 9.1657096479339e-05, 'r': 8, 'lora_alpha': 59}. Best is trial 4 with value: 0.5597586424127885.


In [ ]:
# prompt: #[I 2025-05-13 20:24:40,021] Trial 4 finished with value: 0.5597586424127885 and parameters: {'learning_rate': 9.1657096479339e-05, 'r': 8, 'lora_alpha': 59}. Best is trial 4 with value: 0.5597586424127885.
# convierte junto con params twitter  aun diccionario llamado optimized_2, tienes que inlcuir los parms twitter tambie

optimized_2 = {
    'num_train_epochs': 100,
    'learning_rate': 9.1657096479339e-05,
    'per_device_train_batch_size': 8,
    'per_device_eval_batch_size': 8,
    'logging_steps': 100,
    'r': 8,
    'lora_alpha': 59
}


In [ ]:
final_model_2, final_metrics_2 = sexism_classification_pipeline_task2_LoRA(
    EnTrainTask2, EnDevTask2, None,
    best_model_name, 4, "single_label_classification",
    **optimized_2
)


In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 1. Dataset for Task 2
class EnTestDatasetTask2(Dataset):
    def __init__(self, ids, texts, tokenizer, max_length=128):
        self.ids = ids
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        item_id = self.ids[idx]
        text = self.texts[idx]
        enc = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        batch = {k: v.squeeze(0) for k, v in enc.items()}
        batch["id"] = item_id
        return batch

# 2. Load tokenizer + model (4 classes)
modelname = best_model_name  # e.g. "cardiffnlp/twitter-roberta-base-2022-154m"
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForSequenceClassification.from_pretrained(
    "./final_best_model_mixpeft_2",
    num_labels=4
)
model.eval()
model.cuda()

# 3. Prepare DataLoader
ids_list, texts_list = EnTestTask2[0], EnTestTask2[1]
dataset = EnTestDatasetTask2(ids_list, texts_list, tokenizer)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# 4. Inference: collect both hard and soft
hard_results = []
soft_results = []
softmax = torch.nn.Softmax(dim=-1)
label_map = {0: "NO", 1: "DIRECT", 2: "REPORTED", 3: "JUDGEMENTAL"}

with torch.no_grad():
    for batch in loader:
        input_ids = batch["input_ids"].cuda()
        attention_mask = batch["attention_mask"].cuda()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        # Compute probabilities and hard predictions
        probs_tensor = softmax(outputs.logits)          # [B, 4]
        preds_tensor = outputs.logits.argmax(dim=-1)    # [B]

        probs = probs_tensor.cpu().tolist()             # list of [p0, p1, p2, p3]
        preds = preds_tensor.cpu().tolist()             # list of ints in {0,1,2,3}

        for item_id, prob_vec, pred in zip(batch["id"], probs, preds):
            # Hard output
            hard_results.append({
                "test_case": "EXIST2025",
                "id": item_id,
                "value": label_map[pred]
            })
            # Soft output
            soft_results.append({
                "test_case": "EXIST2025",
                "id": item_id,
                "value": {
                    "NO":           prob_vec[0],
                    "DIRECT":       prob_vec[1],
                    "REPORTED":     prob_vec[2],
                    "JUDGEMENTAL":  prob_vec[3]
                }
            })

# 5. Save both JSONs
name = modelname.split("/")[-1]
hard_path = f"predictions_{name}_hard_EnTestTask2.json"
soft_path = f"predictions_{name}_soft_EnTestTask2.json"

with open(hard_path, "w", encoding="utf-8") as f:
    json.dump(hard_results, f, indent=4, ensure_ascii=False)

with open(soft_path, "w", encoding="utf-8") as f:
    json.dump(soft_results, f, indent=4, ensure_ascii=False)

print(f"Saved {len(hard_results)} hard predictions to {hard_path}")
print(f"Saved {len(soft_results)} soft predictions to {soft_path}")


# Task 3

In [ ]:
# prompt: haz un pipeline para la seleccion de un mejor modelo para la task3 probando difernetes repositorios de transformers de hugging face

from functools import partial
from sklearn.preprocessing import MultiLabelBinarizer

class SexismDatasetMulti(Dataset):
    def __init__(self, texts, labels, ids, tokenizer, max_len=128, pad="max_length", trunc=True,rt='pt'):
        self.texts = texts.tolist()
        self.labels = labels
        self.ids = ids
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.pad = pad
        self.trunc = trunc
        self.rt = rt


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,padding=self.pad, truncation=self.trunc,
            return_tensors=self.rt
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float),
            'id': torch.tensor(self.ids[idx], dtype=torch.long)}
# Ejemplo de uso para task 3:

model_names = [
    "cardiffnlp/twitter-roberta-base-2022-154m",
    "cardiffnlp/twitter-roberta-large-2022-154m",
    "cardiffnlp/twitter-xlm-roberta-base",
    "cardiffnlp/twitter-roberta-base",
    "sdadas/xlm-roberta-large-twitter",
    "g8a9/distilroberta-base-twitter-16M_aug-oct22",
    "andrea-t94/roberta-fine-tuned-twitter",
    "bdotloh/twitter-roberta-base-finetuned-twitter-user-desc"
]

params_task3 = {
    "num_train_epochs": 10,  # Ajusta según sea necesario
    "learning_rate": 5e-5,   # Ajusta según sea necesario
    "per_device_train_batch_size": 8,
    "per_device_eval_batch_size": 16,
    "logging_steps": 10,
    "early_stopping_patience": 3
}


df_metrics_task3 = run_lora_experiments(
    task_num=3,
    model_names=model_names,
    params=params_task3,
    trainInfo=EnTrainTask3,
    devInfo=EnDevTask3,
    testInfo=None  # Configura testInfo si tienes datos de test
)

print(df_metrics_task3)

best_model_task3 = select_best_model(df_metrics_task3, 3)
print(f"→ Best model for task 3: {best_model_task3}")




In [ ]:

optimized_task3 = optimize_lora_hyperparams(
    task_num=3,
    best_model_name=best_model_task3,
    params_base=params_task3,
    trainInfo=EnTrainTask3,
    devInfo=EnDevTask3,
    n_trials=5 # Reduce el número de trials para pruebas
)

print(optimized_task3)

final_model_task3, final_metrics_task3 = sexism_classification_pipeline_task3_LoRA(
    EnTrainTask3, EnDevTask3, None,
    best_model_task3, 5, "multi_label_classification",
    **optimized_task3
)

In [ ]:
resultados=[]
for languaje in ["En", "Sp"]:
  for task in ["1", "2", "3"]:
    params = dict()
    eval(f"incontext_zero_pipeline_task{task}")(model, tokenizer, eval(f"{languaje}DevTask{task}"), eval(f"{languaje}TestTask{task}"), eval(f"output_postprocessing_incontext_zero_s{task}"))
